For my project, I will be locating a possible location for a breakfast spot. I will be exploring hours of operations, proximity to other places that may increase traffic (e.g. churches, hotels), and density of other, competing restaurants. 

In [1]:
import numpy as np
import pandas as pd

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

import folium

The first thing I'll need to do is define the location. I've chosen a midwestern U.S. city, Indianapolis, as the place I will be exploring, so I will need to find information about latitude and longitude. I found a website that will give coordinates for Zip Codes (https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/), so I narrowed it down to Indiana and exported a .csv file. 

In [9]:
inZipRaw = pd.read_csv("us-zip-code-latitude-and-longitude.csv", sep=";")
# We only need Indianapolis 
inCoord = inZipRaw[inZipRaw.City=="Indianapolis"].reset_index(drop=True)
inCoord.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,46227,Indianapolis,IN,39.678495,-86.12973,-5,0,"39.678495,-86.12973"
1,46201,Indianapolis,IN,39.775125,-86.10839,-5,0,"39.775125,-86.10839"
2,46250,Indianapolis,IN,39.905689,-86.06733,-5,0,"39.905689,-86.06733"
3,46228,Indianapolis,IN,39.849474,-86.20448,-5,0,"39.849474,-86.20448"
4,46224,Indianapolis,IN,39.795593,-86.25409,-5,0,"39.795593,-86.25409"


In [10]:
#Visualize

In [11]:
#FourSquare call

In [12]:
#Cluster via type of venue 

In [ ]:
#Cluster via popular operating hours